In [1]:
import t3f
import tensorflow as tf
import numpy as np

In this example we will show how to convert existing tensors into Tensor Train format. Let us consider the tensor obtained by evaluating $\sin(x)$ on a uniform grid of size $2 ^ d$ and reshaping the obtained array into tensor of size $2 \times 2 \times 2 \cdots \times 2$. This is an example of a tensor in the so-called Quantized Tensor Train format (QTT).

In [2]:
d = 10
x = np.linspace(0, 2 * np.pi, 2 ** 10)
y = np.sin(x)
y = np.reshape(y, d * [2], order='f')

In [5]:
print (y.shape)

(2, 2, 2, 2, 2, 2, 2, 2, 2, 2)


Now lets construct a TT tensor out of `y` using `t3f.decompositions.to_tt_tensor`.

In [36]:
y_tt = t3f.decompositions.to_tt_tensor(y, max_tt_rank=32)
print (y_tt)

A Tensor Train of shape (2, 2, 2, 2, 2, 2, 2, 2, 2, 2), TT-ranks: (1, 2, 4, 8, 16, 32, 16, 8, 4, 2, 1)


Let us verify that this indeed an approximation of the original tensor `y`.

In [37]:
with tf.Session() as sess:
    y_tt_full = sess.run(t3f.full(y_tt))
    print ("relative error is {}".format(np.linalg.norm(y_tt_full - y) / np.linalg.norm(y)))

relative error is 2.9627586451526042e-15


It is known that the exact TT-rank of the obtained tensor is equal to $2$. Let us round `y_tt` using `t3f.round` by setting the maximal value of the TT-rank to $2$ and verify it numerically.

In [38]:
y_tt_round = t3f.round(y_tt, max_tt_rank=2)

with tf.Session() as sess:
    y_tt_round_full = sess.run(t3f.full(y_tt_round))
    print ("relative error is {}".format(np.linalg.norm(y_tt_round_full - y) / np.linalg.norm(y)))

relative error is 2.7376238650419183e-15


This fact allows us to significantly compress the tensor `y`. The total number of degrees of freedom in `y_tt` is :

In [39]:
dof = sum(np.prod(tt_core.shape.as_list()) for tt_core in y_tt_round.tt_cores)
print (dof)

72


which is much smaller than the original $2^d$ and is in fact proportional to $d$.

**Exercise**: perform the same analysis for $e^x$.